# Geometry Optimization
In this notebook we will show how to optimize the geometry of a molecule using BigDFT.

Author note: this is related to some mini-protein calculations I want to do...

## System Setup
We will do an example of OH-. First we define the input positions.

In [1]:
from futile import YamlIO as Y

positions ="""
- O: [-0.931477999671, 0.004547999723, -0.117331000086]
- H: [-1.355753693973, -0.032108553467, 0.725181911626]
"""
posinp = {"positions": Y.load(stream=positions, doc_lists=False),
          "units":"angstroem"}

We might also like to check the length of the bond as we do the optimization.

In [8]:
def oh_length(posinp):
    from numpy import array
    from numpy.linalg import norm
    
    pos1 = posinp["positions"][0]
    pos1 = array([float(x) for x in list(pos1.items())[0][1]])
    pos2 = posinp["positions"][1]
    pos2 = array([float(x) for x in list(pos2.items())[0][1]])
    
    return norm(pos2 - pos1)

In [9]:
print(oh_length(posinp))

0.9440241384989304


## Calculation Conditions
Now we will setup the input file for geometry optimization.

In [10]:
from BigDFT import Inputfiles as I
inp = I.Inputfile()
inp.set_xc("PBE")
inp.set_hgrid(0.4)
inp.write_orbitals_on_disk()

We will give this run a background charge to balance out the negative ion.

In [11]:
inp["dft"]["qcharge"] = -1.0

Finally, the parameters for optimizing the geometry.

In [12]:
inp["geopt"] = {}
inp["geopt"]["method"] = "FIRE"

And a calculator to run the calculation.

In [15]:
from BigDFT import Calculators as C
code = C.SystemCalculator()

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -machinefile /var/lib/oar/1883596 /home/wdawson/share/binaries/python3/install/bin/bigdft


## Actual Run and Convergence
Now we will do the actual run and check the convergence.

In [16]:
result = code.run(name="opt", input=inp, posinp=posinp, run_dir="geom")

Creating the yaml input file "geom/opt.yaml"
Run directory geom
Executing command:  mpirun -machinefile /var/lib/oar/1883596 /home/wdawson/share/binaries/python3/install/bin/bigdft -n opt
Found 9 different runs


In [17]:
try:
    result.geopt_plot()
except:
    print("No image support")

<Figure size 640x480 with 1 Axes>

We can check the new bond length.

In [18]:
opt_length = oh_length(result.log["Atomic structure"])
print(opt_length)

0.9787642972453493
